## How to use LangChain and Azure OpenAI with Python
##### Langchain is an open source framework for developing applications using large language models (LLM).
##### This guide will demonstrate how to setup and use Azure OpenAI models' API with LangChain.

### Set Up
##### The following libraries must be installed to use LangChain with Azure OpenAI.

In [ ]:
#%pip install --upgrade openai
#%pip install langchain

#Update langchain for openai
#%pip install -U langchain-openai

### API Configuation and Deployed Model Setup
##### After installing the necessary libraies, the API must be configured. The code below shows how to configure the API directly in your Python environment.

### Deployed Model Setup

In [1]:
import json
import os
#from openai import AzureOpenAI
from langchain.chat_models import AzureChatOpenAI
from langchain_openai import AzureChatOpenAI
from langchain.schema import HumanMessage
from langchain import LLMChain


In [2]:

!python -V
!openai -V
!langchain --version -v

Python 3.11.5
openai 1.12.0
langchain-cli 0.0.20


In [3]:
# Create an instance of chat llm

llm = AzureChatOpenAI(
    openai_api_version="2023-07-01-preview",
    azure_deployment="gpt-35-turbo-16k",
)

llm.invoke([HumanMessage(content="Write me a poem")])

AIMessage(content="In the stillness of a moonlit night,\nWhere dreams take flight in gentle light,\nI'll weave a tale with words so fine,\nA poem that dances in your mind.\n\nLike whispers of a secret breeze,\nThis verse will bring you to your knees,\nA symphony of words, soft and sweet,\nTo make your heart skip a gentle beat.\n\nThrough rolling meadows, fields so green,\nWhere sunrays kiss the daisy's sheen,\nI'll paint a picture, vivid and bright,\nOf nature's beauty, a sheer delight.\n\nThe ocean's roar, a soothing sound,\nAs waves crash upon the sandy ground,\nI'll capture its essence, wild and free,\nA rhythm that echoes eternally.\n\nIn love's embrace, I'll let you dwell,\nA tender story I will tell,\nOf hearts entwined with passion's fire,\nAn eternal flame that won't expire.\n\nBut as all tales must reach an end,\nI'll bid farewell, my dear friend,\nKnow that these words were meant for you,\nA piece of my soul, forever true.\n\nSo let this poem be your guide,\nThrough joy and s

### Prompt Templates
##### Langchain provides a built in PromptsTemplate module to simplify the construction of prompts to get more specific answers.

In [5]:
from langchain import PromptTemplate

from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
)


# First Example
template = """
You are a skin care consulant that recommends products based on customer
needs and preferences.

What is a good {product_type} to help with {customer_request}?
"""

prompt = PromptTemplate(
input_variables=["product_type", "customer_request"],
template=template,
)

print("Example #1:")
print(llm.invoke([HumanMessage(content=prompt.format(
        product_type="face wash",
        customer_request = "acne prone skin"
    ))]
))
print("\n")

# Second Example
system_message = "You are an AI assistant travel assistant that provides vacation recommendations."

system_message_prompt = SystemMessagePromptTemplate.from_template(system_message)
human_template="{text}"
human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)
chat_prompt = ChatPromptTemplate.from_messages([system_message_prompt, human_message_prompt])
chain = LLMChain(llm=llm, prompt=chat_prompt)
result = chain.invoke(f"Where should I go on vaction in Decemember for warm weather and beaches?")
print("Example #2:")
print(result)

Example #1:
content="A good face wash for acne-prone skin is one that effectively cleanses the skin, removes excess oil and impurities, and helps to prevent breakouts. Some key ingredients to look for in a face wash for acne-prone skin include salicylic acid, benzoyl peroxide, tea tree oil, and witch hazel. These ingredients have proven to be effective in treating acne and reducing inflammation.\n\nOne popular face wash for acne-prone skin is the CeraVe Acne Foaming Cream Cleanser. It contains 4% benzoyl peroxide, which helps to clear acne and prevent future breakouts. It is also non-comedogenic, meaning it won't clog pores, and it is gentle enough for daily use.\n\nAnother recommended option is the Neutrogena Oil-Free Acne Wash. This face wash contains salicylic acid, which helps to unclog pores and exfoliate the skin. It effectively removes dirt, oil, and makeup without over-drying the skin.\n\nIt is important to note that everyone's skin is unique, so what works for one person may n

### Chains
##### There are many applications of chains that allow you to combine numerous LLM calls and actions.

### Simple Sequential Chains
##### Allow you to feed the output of one LLM Chain as input for another.

In [6]:
from langchain.chains import SimpleSequentialChain

In [7]:
description_template = """Your job is to come up with a fun DIY project for the specified gender, age, and description of a kid.
% CHILD_DESCRIPTION
{child_description}

YOUR RESPONSE:
"""
description_prompt_template = PromptTemplate(input_variables=["child_description"], template=description_template)

description_chain = LLMChain(llm=llm, prompt=description_prompt_template)

In [8]:
diy_description_template = """Given a DIY project, give a short and simple recipe step-by-step guide on how to complete the project and a materials list.
% DIY_PROJECT
{diy_project}

YOUR RESPONSE:
"""
diy_prompt_template = PromptTemplate(input_variables=["diy_project"], template=diy_description_template)

diy_chain = LLMChain(llm=llm, prompt=diy_prompt_template)

In [9]:
overall_chain = SimpleSequentialChain(chains=[description_chain, diy_chain], verbose=True)

In [11]:
review = overall_chain.invoke("5-year-old girl")



> Entering new SimpleSequentialChain chain...
DIY Princess Crown: 
Materials needed: 
- Cardboard
- Glitter
- Craft glue
- Scissors
- Decorative gems or stickers
- Elastic string or ribbon

Instructions:
1. Start by cutting a strip of cardboard long enough to fit around the child's head. Make sure to leave a little extra length for overlapping.
2. Measure the strip around the child's head to ensure it fits comfortably, then glue the ends together to form a circular shape.
3. Cut out a crown shape from the cardboard, making sure it has points or spikes at the top.
4. Decorate the crown using craft glue and glitter. You can cover the entire crown in glitter or create patterns and designs.
5. Allow the glue to dry completely before moving on to the next step.
6. Once the glitter is dry, add decorative gems or stickers to the crown for extra sparkle and flair.
7. Finally, attach elastic string or ribbon to the ends of the crown, creating a loop that can be worn around the child's head.
8